# Momento de Retroalimentación: Módulo 2 Uso de framework o biblioteca de aprendizaje máquina para la implementación de una solución

Jorge Eduardo de León Reyna - A00829759

## Carga de librerias

In [39]:
import pandas as pd
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

## Descripcion del dataset utilizado

- **Dataset utilizado:** Heart Failure Prediction
- **Descripción y contexto del problema a resolver:** Las enfermedades cardiovasculares son la principal causa de muerte en todo el mundo, cobrando cerca de 17.9 millones de vidas al año y representando el 31% de todas las muertes. La mayoría de estas muertes son por ataques cardíacos y accidentes cerebrovasculares, y muchas ocurren prematuramente en personas menores de 70 años. La insuficiencia cardíaca también es común debido a estas enfermedades. Un conjunto de datos con 11 características puede ayudar a predecir posibles problemas cardíacos. Las personas con enfermedades cardíacas o alto riesgo cardiovascular, debido a factores como hipertensión o diabetes, necesitan detección temprana y manejo, donde un modelo de aprendizaje automático puede ser muy útil.
- **Atributos o campos:**
  - Age: edad del paciente
      - años
  - Sex: sexo del paciente
      - M: Male, F: Female
  - ChestPainType: tipo de dolor de pecho
      - TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic
  - RestingBP: presión arterial en reposo
      - mm Hg
  - Cholesterol: colesterol
      - mm/dl
  - FastingBS: azucar en la sangre en ayunas
      - 1: si FastingBS > 120 mg/dl, 0: otro
  - RestingECG: resultados de electrocardiograma en reposo
      - Normal: Normal, ST: Anomalía de la onda ST-T (inversiones de onda T y/o elevación o depresión del segmento ST > 0.05 mV), LVH: mostrando hipertrofia ventricular izquierda probable o definitiva según los criterios de Estes.
  - MaxHR: frecuencia cardíaca máxima alcanzada
      - Valor numero entre 60 y 202
  - ExerciseAngina: angina inducida por el ejercicio
      - Y: Yes, N: No
  - Oldpeak: Descenso antiguo = ST
      - Valor numerico
  - ST_Slope: La pendiente del segmento ST durante el ejercicio máximo
      - Up: upsloping, Flat: flat, Down: downsloping
  - HeartDisease: salida
      - 1: heart disease, 0: Normal
- **Enlace:** https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction?resource=download
- **Descripcion del problema:** En este dataset se busca hacer un modelo de clasificación, donde se tienen dos clases posibles para predecir: "heart disease(1)" y "normal(1)".
- **Cantidad de registros:** 918








## Cargando Dataset

In [40]:
df = pd.read_csv("/content/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [41]:
# obteniendo cantidad de registros
len(df)

918

#### Limpieza de datos inicial

In [42]:
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True)

## Matriz de correlación
Se busca encontrar las variables con mayor influencia sobre la variable objetivo ("charges") para utilizar las mismas en las pruebas del modelo.

In [43]:
correlation_matrix = df.corr()
print(correlation_matrix)

                   Age  RestingBP  Cholesterol  FastingBS     MaxHR   Oldpeak  \
Age           1.000000   0.254399    -0.095282   0.198039 -0.382045  0.258612   
RestingBP     0.254399   1.000000     0.100893   0.070193 -0.112135  0.164803   
Cholesterol  -0.095282   0.100893     1.000000  -0.260974  0.235792  0.050148   
FastingBS     0.198039   0.070193    -0.260974   1.000000 -0.131438  0.052698   
MaxHR        -0.382045  -0.112135     0.235792  -0.131438  1.000000 -0.160691   
Oldpeak       0.258612   0.164803     0.050148   0.052698 -0.160691  1.000000   
HeartDisease  0.282039   0.107589    -0.232741   0.267291 -0.400421  0.403951   

              HeartDisease  
Age               0.282039  
RestingBP         0.107589  
Cholesterol      -0.232741  
FastingBS         0.267291  
MaxHR            -0.400421  
Oldpeak           0.403951  
HeartDisease      1.000000  


<ipython-input-43-becb50483c66>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


Se encuentra que las variables con mayor correlacion a la variable objetivo ("charges") son: **"Oldpeak", "Age" y "FastingBS"** por lo que seran las seleccionadas para realizar pruebas con el modelo de regresion logistica.

## Preprocesamiento de datos y encoding

In [44]:
# Dividiendo dataset en features & lables
x = df[["Age", "Sex",	"ChestPainType", "RestingBP",	"Cholesterol",	"FastingBS",	"RestingECG", "MaxHR",	"ExerciseAngina",	"Oldpeak", "ST_Slope"]]
y = df["HeartDisease"]

# Definimos variables categoricos y numericos
categorical_features = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope" ]
numerical_features = ["Age","RestingBP","Cholesterol","FastingBS","MaxHR","Oldpeak"]


In [45]:

# Definimos pipeline para onehot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
    ],
    remainder='passthrough'
)

# Aplicamos one-hot encoding
x_encoded = preprocessor.fit_transform(x)

## Dividiendo dataset en set de entrenamiento y set de pruebas

In [46]:
# Dividimos datos de entrenamiento y en 3 subsets de pruebas
x_train, x_test1, y_train, y_test1 = train_test_split(x_encoded, y, test_size=0.3, random_state=42)
x_test2, x_test3, y_test2, y_test3 = train_test_split(x_test1, y_test1, test_size=0.5, random_state=42)



---



## Pipeline de entrenamiento y modificacion de hiperparametros

In [47]:
# Definimos  pipeline de entrenamiento y modificamos hiperparametros
model = Pipeline(steps=[
    ('classifier', LogisticRegression(
        penalty='l2', # Tipo de penalizacion
        C=1.0, # Parmetro de regularización
        solver='liblinear', # Solucionador
        max_iter=10000 # Numero de iteraciones
    ))
])

##Ejecucion de entrenamiento del modelo

In [48]:
model.fit(x_train, y_train)

Pipeline(steps=[('classifier',
                 LogisticRegression(max_iter=10000, solver='liblinear'))])

## Generacion de predicciones y evaluacion del modelo

### Prueba y comparacion con valores puntuales distintos a los de entrenamiento

In [49]:
# Seleccionamos valores puntuales del dataset diferentes al set de entrenamiento
indices_prueba = [145, 23, 250, 622, 457]
x_samples = x_train[indices_prueba]
y_samples = y_train.iloc[indices_prueba].values

# Usamos los datos seleccionados de prueba para hacer las predicciones y comparar
predicted_samples = model.predict(x_samples)

# Comparacion de los valores reales contra los predichos
for i in range(len(indices_prueba)):
    print(f"Muestra {i + 1}:")
    print(f"Valor real: {y_samples[i]}")
    print(f"Valor predicho: {predicted_samples[i]}")
    print()

Muestra 1:
Valor real: 1
Valor predicho: 1

Muestra 2:
Valor real: 0
Valor predicho: 0

Muestra 3:
Valor real: 1
Valor predicho: 1

Muestra 4:
Valor real: 0
Valor predicho: 0

Muestra 5:
Valor real: 1
Valor predicho: 1



### Descripcion de metricas de evaluacion para set de entrenamiento

Para estimar qué tan bueno es cada clasificador, debemos generar las matrices de confusión y las correspondientes métricas:

$accuracy = \frac{VP+VN}{VP+VN+FP+FN}$
- Accuracy (Exactitud): Calcula la proporción de predicciones correctas totales del modelo en relación con el número total de predicciones. Es útil cuando las clases están balanceadas.

$precision = \frac{VP}{VP+FP}$
- Precision (Precisión): Evalúa la proporción de instancias positivas que el modelo clasifica correctamente en relación con todas las instancias que predijo como positivas. Es especialmente importante cuando los falsos positivos son costosos o problemáticos.

$recall = \frac{VP}{VP+FN}$
- Recall (Recuerdo o Sensibilidad): Mide la proporción de positivos reales que el modelo identifica correctamente en relación con todos los positivos reales presentes en los datos.


$F1 = \frac{2\cdot precision \cdot recall}{precision+recall}$
- F1 Score: Es una métrica que combina precisión y recall para proporcionar una medida equilibrada del rendimiento del modelo, especialmente cuando hay un desequilibrio entre las clases.




### Evaluacion de modelo con subset de entrenamiento

In [50]:
# Predicciones con
y_pred1 = model.predict(x_train)

In [51]:
# Matriz de confusión
confusion = confusion_matrix(y_train, y_pred1)
print("Matriz de Confusión prueba 1:")
print(confusion)

Matriz de Confusión prueba 1:
[[247  51]
 [ 38 306]]


In [52]:
# Calculo de accuracy
accuracy = accuracy_score(y_train, y_pred1)
print("Accuracy prueba 1:", accuracy)

Accuracy prueba 1: 0.8613707165109035


In [53]:
# Reporte de métricas (precision, recall, f1-score, support)
report = classification_report(y_train, y_pred1)
print("Reporte de Clasificación Prueba 1:")
print(report)

Reporte de Clasificación Prueba 1:
              precision    recall  f1-score   support

           0       0.87      0.83      0.85       298
           1       0.86      0.89      0.87       344

    accuracy                           0.86       642
   macro avg       0.86      0.86      0.86       642
weighted avg       0.86      0.86      0.86       642



### Descripcion de metricas de evaluacion para set de prueba

Para estimar qué tan bueno es cada clasificador, debemos generar las matrices de confusión y las correspondientes métricas:

$accuracy = \frac{VP+VN}{VP+VN+FP+FN}$
- Accuracy (Exactitud): Calcula la proporción de predicciones correctas totales del modelo en relación con el número total de predicciones. Es útil cuando las clases están balanceadas.

$precision = \frac{VP}{VP+FP}$
- Precision (Precisión): Evalúa la proporción de instancias positivas que el modelo clasifica correctamente en relación con todas las instancias que predijo como positivas. Es especialmente importante cuando los falsos positivos son costosos o problemáticos.

$recall = \frac{VP}{VP+FN}$
- Recall (Recuerdo o Sensibilidad): Mide la proporción de positivos reales que el modelo identifica correctamente en relación con todos los positivos reales presentes en los datos.


$F1 = \frac{2\cdot precision \cdot recall}{precision+recall}$
- F1 Score: Es una métrica que combina precisión y recall para proporcionar una medida equilibrada del rendimiento del modelo, especialmente cuando hay un desequilibrio entre las clases.




### Evaluacion de modelo con subset de prueba

In [54]:
# Predicciones con
y_pred1 = model.predict(x_test1)

In [55]:
# Matriz de confusión
confusion = confusion_matrix(y_test1, y_pred1)
print("Matriz de Confusión prueba 1:")
print(confusion)

Matriz de Confusión prueba 1:
[[100  12]
 [ 20 144]]


In [56]:
# Calculo de accuracy
accuracy = accuracy_score(y_test1, y_pred1)
print("Accuracy prueba 1:", accuracy)

Accuracy prueba 1: 0.8840579710144928


In [57]:
# Reporte de métricas (precision, recall, f1-score, support)
report = classification_report(y_test1, y_pred1)
print("Reporte de Clasificación Prueba 1:")
print(report)

Reporte de Clasificación Prueba 1:
              precision    recall  f1-score   support

           0       0.83      0.89      0.86       112
           1       0.92      0.88      0.90       164

    accuracy                           0.88       276
   macro avg       0.88      0.89      0.88       276
weighted avg       0.89      0.88      0.88       276



### Analisis de resultados de las metricas de evaluacion
 Despues de calcular los valores de la matriz de confusión para cada una de las implementaciones del modelo y el posterior calculo de las metricas de evaluacion en base a estas se encontró lo siguiente:
 1. Al haccer pruebas tanto con el set de pruebas como con el de entrenamiento se encontró que el desempeño de ambos no indica ni overfitting ni underfitting al tener niveles de error similar, lo cual indica un desempeño adecuado del modelo.

 2. Al hacer las pruebas finales con el set de pruebas, se obtuvieron las siguientes metricas de evaluacion cuyo impacto en el desempeño del modelo se describen a continuación: (en el reporte generado por sklearn se toma en cuenta la seccion de "wighted avg" dado el contexto del problema a resolver donde la precision del resultado en la clasificacion es muy importante):

  - **Accuracy:** 0.88, lo cual nos indican un buen funcionamiento correcto del modelo a primera instancia.
  - **Precision:**  0.89, lo que nos indica una buen nivel de prediccion en los valores positivos (1's) en comparacion a todas las muestras positivas (verdaderas y falsas).
  - **Recall:** 0.88, lo que nos indica una buen nivel de prediccion en los valores positivos verdaderos en comparacion al total de muestras positivas verdaderas (1's).
  - **F1-score:** 0.88, lo que nos indica un desempeño adecuado a nivel general a ser la medida armonica entre precision y recall.